In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from utils.DataUtils import DataUtils
from Processing import spectrogram
import os
import json
from SignalPreProcessor import normalize

def main():
    data = ['adbecg', 'nifecg', 'synt_ecg']
    data_utils = DataUtils()

    # # ADBECG
    adbecg = data_utils.get_data(dataset='adbecg')
    # # # NIFECG
    # nifecg = data_utils.get_data(dataset='nifecg')
    # # SYNT_ECG
    # synt_ecg = data_utils.get_data(dataset='synt_ecg')
    
    fs = 1000  # 1000Hz sampling frequency
    
    segment_size = int(0.25 * fs)  # 250ms segment size
    overlap_size = int(0.01 * fs)  # 10ms overlap size
    window = signal.get_window('hamming', segment_size)  # Hamming window
    time_axis = np.arange(0, segment_size) / fs

    # data = {} # create an empty dictionary to store the segments for each record
    # metadata = [] # create an empty list to store the metadata for each record

    for record in adbecg:
        if record.endswith('r01.edf'):
            continue
        record_id = (record.split('/')[-1][1:3])
        print(f"Reading record {record_id}")
        record_signal = data_utils.open_record(record)
        # record_segments = {record_id: {'abdominal': {}, 'fetal': {}}}

        for chn in range(0, len(record_signal.getSampleFrequencies())):
            channel_name = record_signal.getSignalLabels()[chn]
            print('Reading channel: {}'.format(record_signal.getSignalLabels()[chn]))
            _signal = normalize(record_signal.readSignal(chn))
            # _signal = np.interp(_signal, (_signal.min(), _signal.max()), (-1, 1))
            
            segments = []
            segments_index = 0
            for i in range(0, len(_signal) - segment_size, segment_size - overlap_size):
                segment = _signal[i:i+segment_size] * window